In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,log_loss, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
import statsmodels.api as sm

In [85]:
class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [86]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [87]:
X=train.drop(['target','ID_code'],axis=1,inplace=False)
X_test=test.drop('ID_code',axis=1,inplace=False)
y=train['target']

In [88]:
PartA_X, PartB_X, PartA_Y, PartB_Y = train_test_split(X, y, test_size=0.66, random_state=42)
#split the test again to get 33% dev and 33% test
PartB_X, PartC_X, PartB_Y, PartC_Y = train_test_split(PartB_X, PartB_Y, test_size=0.5, random_state=42)

In [89]:
XGB_PartA_X = xgb.DMatrix(PartA_X, PartA_Y)
XGB_PartB_X = xgb.DMatrix(PartB_X, PartB_Y)
XGB_PartC_X= xgb.DMatrix(PartC_X, PartC_Y)

In [8]:
params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": 0.2,
        "tree_method": 'exact',
        "max_depth": 17,
        "subsample": 1,
        "colsample_bytree": 1,
        "silent": 1,
        "min_chil_weight":1,
        "seed": 42,
        #"num_class" : 22,
    }
num_boost_round = 2000
early_stopping_rounds = 50
watchlist = [(XGB_PartA_X, 'train'), (XGB_PartB_X, 'eval')]

In [9]:
gbm = xgb.train(params, XGB_PartA_X, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)


[0]	train-auc:0.815357	eval-auc:0.626386
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.892997	eval-auc:0.675602
[2]	train-auc:0.931484	eval-auc:0.695948
[3]	train-auc:0.955434	eval-auc:0.706716
[4]	train-auc:0.968432	eval-auc:0.715722
[5]	train-auc:0.981702	eval-auc:0.723284
[6]	train-auc:0.988312	eval-auc:0.729455
[7]	train-auc:0.992777	eval-auc:0.734907
[8]	train-auc:0.995687	eval-auc:0.73978
[9]	train-auc:0.997694	eval-auc:0.743848
[10]	train-auc:0.99853	eval-auc:0.749144
[11]	train-auc:0.999015	eval-auc:0.755822
[12]	train-auc:0.999582	eval-auc:0.759767
[13]	train-auc:0.999735	eval-auc:0.764258
[14]	train-auc:0.99992	eval-auc:0.767642
[15]	train-auc:0.999976	eval-auc:0.771436
[16]	train-auc:0.999995	eval-auc:0.775211
[17]	train-auc:0.999996	eval-auc:0.777753
[18]	train-auc:0.999998	eval-auc:0.780586
[19]	train-auc:1	eval-auc:0.783119
[20]	train-auc:1	eval-auc:0.786821
[21]

[224]	train-auc:1	eval-auc:0.867679
[225]	train-auc:1	eval-auc:0.867711
[226]	train-auc:1	eval-auc:0.867777
[227]	train-auc:1	eval-auc:0.867882
[228]	train-auc:1	eval-auc:0.86792
[229]	train-auc:1	eval-auc:0.86793
[230]	train-auc:1	eval-auc:0.867953
[231]	train-auc:1	eval-auc:0.868041
[232]	train-auc:1	eval-auc:0.868075
[233]	train-auc:1	eval-auc:0.868114
[234]	train-auc:1	eval-auc:0.868126
[235]	train-auc:1	eval-auc:0.868126
[236]	train-auc:1	eval-auc:0.868131
[237]	train-auc:1	eval-auc:0.868186
[238]	train-auc:1	eval-auc:0.868207
[239]	train-auc:1	eval-auc:0.868296
[240]	train-auc:1	eval-auc:0.868297
[241]	train-auc:1	eval-auc:0.868429
[242]	train-auc:1	eval-auc:0.868512
[243]	train-auc:1	eval-auc:0.868518
[244]	train-auc:1	eval-auc:0.8686
[245]	train-auc:1	eval-auc:0.868654
[246]	train-auc:1	eval-auc:0.868704
[247]	train-auc:1	eval-auc:0.868764
[248]	train-auc:1	eval-auc:0.868878
[249]	train-auc:1	eval-auc:0.868881
[250]	train-auc:1	eval-auc:0.868978
[251]	train-auc:1	eval-auc:0.869

[453]	train-auc:1	eval-auc:0.874524
[454]	train-auc:1	eval-auc:0.874564
[455]	train-auc:1	eval-auc:0.874576
[456]	train-auc:1	eval-auc:0.874604
[457]	train-auc:1	eval-auc:0.874628
[458]	train-auc:1	eval-auc:0.874669
[459]	train-auc:1	eval-auc:0.874661
[460]	train-auc:1	eval-auc:0.874676
[461]	train-auc:1	eval-auc:0.874681
[462]	train-auc:1	eval-auc:0.874695
[463]	train-auc:1	eval-auc:0.874686
[464]	train-auc:1	eval-auc:0.874697
[465]	train-auc:1	eval-auc:0.874709
[466]	train-auc:1	eval-auc:0.874717
[467]	train-auc:1	eval-auc:0.874741
[468]	train-auc:1	eval-auc:0.874765
[469]	train-auc:1	eval-auc:0.87478
[470]	train-auc:1	eval-auc:0.874818
[471]	train-auc:1	eval-auc:0.874822
[472]	train-auc:1	eval-auc:0.87482
[473]	train-auc:1	eval-auc:0.874823
[474]	train-auc:1	eval-auc:0.874854
[475]	train-auc:1	eval-auc:0.874848
[476]	train-auc:1	eval-auc:0.874852
[477]	train-auc:1	eval-auc:0.874882
[478]	train-auc:1	eval-auc:0.87491
[479]	train-auc:1	eval-auc:0.874945
[480]	train-auc:1	eval-auc:0.87

[682]	train-auc:1	eval-auc:0.876444
[683]	train-auc:1	eval-auc:0.876443
[684]	train-auc:1	eval-auc:0.876474
[685]	train-auc:1	eval-auc:0.876488
[686]	train-auc:1	eval-auc:0.876468
[687]	train-auc:1	eval-auc:0.876463
[688]	train-auc:1	eval-auc:0.876469
[689]	train-auc:1	eval-auc:0.876475
[690]	train-auc:1	eval-auc:0.876484
[691]	train-auc:1	eval-auc:0.876496
[692]	train-auc:1	eval-auc:0.876511
[693]	train-auc:1	eval-auc:0.876524
[694]	train-auc:1	eval-auc:0.87652
[695]	train-auc:1	eval-auc:0.876552
[696]	train-auc:1	eval-auc:0.876569
[697]	train-auc:1	eval-auc:0.876561
[698]	train-auc:1	eval-auc:0.876591
[699]	train-auc:1	eval-auc:0.8766
[700]	train-auc:1	eval-auc:0.876612
[701]	train-auc:1	eval-auc:0.876608
[702]	train-auc:1	eval-auc:0.876633
[703]	train-auc:1	eval-auc:0.876618
[704]	train-auc:1	eval-auc:0.876616
[705]	train-auc:1	eval-auc:0.876607
[706]	train-auc:1	eval-auc:0.876622
[707]	train-auc:1	eval-auc:0.876628
[708]	train-auc:1	eval-auc:0.876638
[709]	train-auc:1	eval-auc:0.87

[911]	train-auc:1	eval-auc:0.877477
[912]	train-auc:1	eval-auc:0.877465
[913]	train-auc:1	eval-auc:0.877482
[914]	train-auc:1	eval-auc:0.877488
[915]	train-auc:1	eval-auc:0.877478
[916]	train-auc:1	eval-auc:0.877498
[917]	train-auc:1	eval-auc:0.877496
[918]	train-auc:1	eval-auc:0.877506
[919]	train-auc:1	eval-auc:0.877511
[920]	train-auc:1	eval-auc:0.8775
[921]	train-auc:1	eval-auc:0.877496
[922]	train-auc:1	eval-auc:0.877501
[923]	train-auc:1	eval-auc:0.877497
[924]	train-auc:1	eval-auc:0.877502
[925]	train-auc:1	eval-auc:0.877509
[926]	train-auc:1	eval-auc:0.877509
[927]	train-auc:1	eval-auc:0.877514
[928]	train-auc:1	eval-auc:0.877519
[929]	train-auc:1	eval-auc:0.877515
[930]	train-auc:1	eval-auc:0.877519
[931]	train-auc:1	eval-auc:0.877525
[932]	train-auc:1	eval-auc:0.877512
[933]	train-auc:1	eval-auc:0.877503
[934]	train-auc:1	eval-auc:0.877486
[935]	train-auc:1	eval-auc:0.877497
[936]	train-auc:1	eval-auc:0.877499
[937]	train-auc:1	eval-auc:0.877512
[938]	train-auc:1	eval-auc:0.8

In [90]:
y_pred_PartB_XGB = gbm.predict(xgb.DMatrix(PartB_X))
y_pred_PartC_XGB = gbm.predict(xgb.DMatrix(PartC_X))
y_pred_Xtest_XGB = gbm.predict(xgb.DMatrix(X_test))

In [91]:
PartB_Y_after_XGB_NN=pd.DataFrame(PartB_Y)
PartC_Y_after_XGB_NN=pd.DataFrame(PartC_Y)

In [159]:
PartB_Y_after_XGB_NN['XGB_predict']=y_pred_PartB_XGB
PartC_Y_after_XGB_NN['XGB_predict']=y_pred_PartC_XGB

#### NN

In [107]:
scaller=MinMaxScaler()
PartA_X=scaller.fit_transform(PartA_X)
PartB_X=scaller.fit_transform(PartB_X)
PartC_X=scaller.fit_transform(PartC_X)
X_test=scaller.fit_transform(X_test)

In [109]:
model = Sequential()
model.add(Dense(500, input_dim=200))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(500))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))    

In [110]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(PartA_Y),PartA_Y)

In [111]:
class_weights

array([0.55540128, 5.01253133])

In [112]:
learning_rate = 0.001
epochs=1000
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, decay=decay_rate, nesterov=False)

In [113]:
model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

In [114]:
filepath="model_stacking.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max', save_weights_only=False)

In [115]:
earlystopping=EarlyStopping(monitor='val_acc', min_delta=0.000001, patience=50,  verbose=0, mode='max', restore_best_weights=True)


In [116]:
roc_auc=roc_auc_callback(training_data=(PartA_X, PartA_Y),validation_data=(PartB_X, PartB_Y))

In [117]:
model.fit(PartA_X, PartA_Y,validation_data=(PartB_X, PartB_Y), epochs=epochs, batch_size=100,verbose=2,
          callbacks=[checkpoint,earlystopping,roc_auc],class_weight=class_weights)

Train on 68000 samples, validate on 66000 samples
Epoch 1/1000
 - 8s - loss: 0.7405 - acc: 0.5866 - val_loss: 0.5142 - val_acc: 0.7722

Epoch 00001: val_acc improved from -inf to 0.77221, saving model to model_stacking.h5
roc_auc: 0.60601 - roc_auc_val: 0.58916 - norm_gini: 0.21202 - norm_gini_val: 0.17831          
Epoch 2/1000
 - 7s - loss: 0.5444 - acc: 0.7299 - val_loss: 0.4115 - val_acc: 0.8679

Epoch 00002: val_acc improved from 0.77221 to 0.86794, saving model to model_stacking.h5
roc_auc: 0.66969 - roc_auc_val: 0.65014 - norm_gini: 0.33938 - norm_gini_val: 0.30029          
Epoch 3/1000
 - 7s - loss: 0.4521 - acc: 0.8068 - val_loss: 0.3576 - val_acc: 0.8925

Epoch 00003: val_acc improved from 0.86794 to 0.89245, saving model to model_stacking.h5
roc_auc: 0.71035 - roc_auc_val: 0.68835 - norm_gini: 0.42071 - norm_gini_val: 0.3767          
Epoch 4/1000
 - 7s - loss: 0.4040 - acc: 0.8425 - val_loss: 0.3262 - val_acc: 0.8981

Epoch 00004: val_acc improved from 0.89245 to 0.89809, 

 - 7s - loss: 0.2494 - acc: 0.9091 - val_loss: 0.2483 - val_acc: 0.9091

Epoch 00032: val_acc did not improve from 0.90917
roc_auc: 0.87877 - roc_auc_val: 0.84665 - norm_gini: 0.75754 - norm_gini_val: 0.69331          
Epoch 33/1000
 - 7s - loss: 0.2468 - acc: 0.9098 - val_loss: 0.2483 - val_acc: 0.9092

Epoch 00033: val_acc improved from 0.90917 to 0.90921, saving model to model_stacking.h5
roc_auc: 0.87969 - roc_auc_val: 0.84706 - norm_gini: 0.75937 - norm_gini_val: 0.69412          
Epoch 34/1000
 - 7s - loss: 0.2481 - acc: 0.9087 - val_loss: 0.2480 - val_acc: 0.9093

Epoch 00034: val_acc improved from 0.90921 to 0.90926, saving model to model_stacking.h5
roc_auc: 0.88042 - roc_auc_val: 0.8476 - norm_gini: 0.76084 - norm_gini_val: 0.69519          
Epoch 35/1000
 - 7s - loss: 0.2481 - acc: 0.9092 - val_loss: 0.2459 - val_acc: 0.9097

Epoch 00035: val_acc improved from 0.90926 to 0.90968, saving model to model_stacking.h5
roc_auc: 0.88128 - roc_auc_val: 0.84858 - norm_gini: 0.76256 -


Epoch 00065: val_acc did not improve from 0.91073
roc_auc: 0.89514 - roc_auc_val: 0.85476 - norm_gini: 0.79028 - norm_gini_val: 0.70953          
Epoch 66/1000
 - 7s - loss: 0.2317 - acc: 0.9144 - val_loss: 0.2458 - val_acc: 0.9105

Epoch 00066: val_acc did not improve from 0.91073
roc_auc: 0.89523 - roc_auc_val: 0.85488 - norm_gini: 0.79047 - norm_gini_val: 0.70975          
Epoch 67/1000
 - 7s - loss: 0.2306 - acc: 0.9144 - val_loss: 0.2457 - val_acc: 0.9105

Epoch 00067: val_acc did not improve from 0.91073
roc_auc: 0.8955 - roc_auc_val: 0.85492 - norm_gini: 0.79099 - norm_gini_val: 0.70984          
Epoch 68/1000
 - 7s - loss: 0.2326 - acc: 0.9141 - val_loss: 0.2453 - val_acc: 0.9106

Epoch 00068: val_acc did not improve from 0.91073
roc_auc: 0.89571 - roc_auc_val: 0.8552 - norm_gini: 0.79142 - norm_gini_val: 0.71039          
Epoch 69/1000
 - 7s - loss: 0.2301 - acc: 0.9144 - val_loss: 0.2470 - val_acc: 0.9104

Epoch 00069: val_acc did not improve from 0.91073
roc_auc: 0.89618 - 

 - 7s - loss: 0.2234 - acc: 0.9174 - val_loss: 0.2460 - val_acc: 0.9109

Epoch 00100: val_acc did not improve from 0.91130
roc_auc: 0.90437 - roc_auc_val: 0.85748 - norm_gini: 0.80875 - norm_gini_val: 0.71496          
Epoch 101/1000
 - 7s - loss: 0.2230 - acc: 0.9173 - val_loss: 0.2464 - val_acc: 0.9110

Epoch 00101: val_acc did not improve from 0.91130
roc_auc: 0.9046 - roc_auc_val: 0.85748 - norm_gini: 0.80919 - norm_gini_val: 0.71495          
Epoch 102/1000
 - 7s - loss: 0.2232 - acc: 0.9167 - val_loss: 0.2450 - val_acc: 0.9114

Epoch 00102: val_acc improved from 0.91130 to 0.91138, saving model to model_stacking.h5
roc_auc: 0.9047 - roc_auc_val: 0.85757 - norm_gini: 0.80939 - norm_gini_val: 0.71513          
Epoch 103/1000
 - 7s - loss: 0.2219 - acc: 0.9173 - val_loss: 0.2467 - val_acc: 0.9110

Epoch 00103: val_acc did not improve from 0.91138
roc_auc: 0.90506 - roc_auc_val: 0.85748 - norm_gini: 0.81013 - norm_gini_val: 0.71495          
Epoch 104/1000
 - 7s - loss: 0.2232 - acc:

 - 7s - loss: 0.2181 - acc: 0.9181 - val_loss: 0.2474 - val_acc: 0.9115

Epoch 00135: val_acc did not improve from 0.91159
roc_auc: 0.9122 - roc_auc_val: 0.85826 - norm_gini: 0.8244 - norm_gini_val: 0.71652          
Epoch 136/1000
 - 8s - loss: 0.2171 - acc: 0.9188 - val_loss: 0.2480 - val_acc: 0.9111

Epoch 00136: val_acc did not improve from 0.91159
roc_auc: 0.91265 - roc_auc_val: 0.85821 - norm_gini: 0.82529 - norm_gini_val: 0.71641          
Epoch 137/1000
 - 8s - loss: 0.2188 - acc: 0.9171 - val_loss: 0.2473 - val_acc: 0.9114

Epoch 00137: val_acc did not improve from 0.91159
roc_auc: 0.91282 - roc_auc_val: 0.85822 - norm_gini: 0.82563 - norm_gini_val: 0.71643          
Epoch 138/1000
 - 8s - loss: 0.2187 - acc: 0.9185 - val_loss: 0.2479 - val_acc: 0.9110

Epoch 00138: val_acc did not improve from 0.91159
roc_auc: 0.91315 - roc_auc_val: 0.85816 - norm_gini: 0.82631 - norm_gini_val: 0.71632          
Epoch 139/1000
 - 7s - loss: 0.2178 - acc: 0.9186 - val_loss: 0.2478 - val_acc: 0


Epoch 00170: val_acc did not improve from 0.91159
roc_auc: 0.92033 - roc_auc_val: 0.85839 - norm_gini: 0.84066 - norm_gini_val: 0.71678          
Epoch 171/1000
 - 7s - loss: 0.2126 - acc: 0.9206 - val_loss: 0.2500 - val_acc: 0.9111

Epoch 00171: val_acc did not improve from 0.91159
roc_auc: 0.92038 - roc_auc_val: 0.85826 - norm_gini: 0.84077 - norm_gini_val: 0.71652          
Epoch 172/1000
 - 8s - loss: 0.2101 - acc: 0.9219 - val_loss: 0.2511 - val_acc: 0.9112

Epoch 00172: val_acc did not improve from 0.91159
roc_auc: 0.9207 - roc_auc_val: 0.8582 - norm_gini: 0.8414 - norm_gini_val: 0.71639          
Epoch 173/1000
 - 8s - loss: 0.2120 - acc: 0.9196 - val_loss: 0.2497 - val_acc: 0.9113

Epoch 00173: val_acc did not improve from 0.91159
roc_auc: 0.92082 - roc_auc_val: 0.85819 - norm_gini: 0.84163 - norm_gini_val: 0.71638          
Epoch 174/1000
 - 8s - loss: 0.2102 - acc: 0.9212 - val_loss: 0.2504 - val_acc: 0.9112

Epoch 00174: val_acc did not improve from 0.91159
roc_auc: 0.9211 

In [118]:
y_pred_PartB_NN = model.predict(PartB_X)
y_pred_PartC_NN = model.predict(PartC_X)
y_pred_Xtest_NN = model.predict(X_test)

In [158]:
PartB_Y_after_XGB_NN['NN_predict']=y_pred_PartB_NN
PartC_Y_after_XGB_NN['NN_predict']=y_pred_PartC_NN

In [121]:
y_pred_Xtest_XGB=pd.DataFrame(y_pred_Xtest_XGB,columns=['Xtest_XGB'])

In [122]:
y_pred_Xtest_XGB['NN_predict_X_test']=y_pred_Xtest_NN

In [123]:
PartC_Y_after_XGB_NN.head(1)

target  XGB_predict_C  NN_predict_C
29667       0       0.000016      0.003368

In [161]:
# PartB_Y_after_XGB_NN
# PartC_Y_after_XGB_NN
# y_pred_Xtest_XGB

part_B_target=PartB_Y_after_XGB_NN['target']
part_C_target=PartC_Y_after_XGB_NN['target']
part_B_X=PartB_Y_after_XGB_NN.drop('target',axis=1,inplace=False)
part_C_X=PartC_Y_after_XGB_NN.drop('target',axis=1,inplace=False)

In [195]:
part_C_X.head()

NN_predict  XGB_predict
29667     0.003368     0.000016
122843    0.078182     0.000358
157682    0.215946     0.000048
198785    0.022033     0.000282
93886     0.382870     0.037677

In [194]:
y_pred_Xtest_XGB.head()

NN_predict  XGB_predict
0    0.000443     0.103653
1    0.001387     0.073271
2    0.013989     0.015402
3    0.019364     0.146464
4    0.000307     0.065677

#### 2nd level models

In [135]:
# 2nd level models
model_2nd = Sequential()
model_2nd.add(Dense(2, input_dim=2))
model_2nd.add(Activation('relu'))
model_2nd.add(BatchNormalization())
#model.add(Dropout(0.2))

model_2nd.add(Dense(2))
model_2nd.add(Activation('relu'))
model_2nd.add(BatchNormalization())
#model.add(Dropout(0.2))

# model.add(Dense(200))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

model_2nd.add(Dense(1))
model_2nd.add(Activation('sigmoid'))  


In [136]:
class_weights2 = class_weight.compute_class_weight('balanced',np.unique(part_B_target),part_B_target)

In [137]:
class_weights2

array([0.55617353, 4.95049505])

In [138]:
model_2nd.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

In [139]:
filepath="model2nd_stacking.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max', save_weights_only=False)

In [140]:
earlystopping=EarlyStopping(monitor='val_acc', min_delta=0.000001, patience=50,  verbose=0, mode='max', restore_best_weights=True)


In [142]:
roc_auc=roc_auc_callback(training_data=(part_B_X, part_B_target),validation_data=(part_C_X, part_C_target))

In [143]:
model_2nd.fit(part_B_X, part_B_target,validation_data=(part_C_X, part_C_target), epochs=epochs, batch_size=100,verbose=2,
          callbacks=[checkpoint,earlystopping,roc_auc],class_weight=class_weights)

Train on 66000 samples, validate on 66000 samples
Epoch 1/1000
 - 2s - loss: 0.7550 - acc: 0.4807 - val_loss: 0.5160 - val_acc: 0.8997

Epoch 00001: val_acc improved from -inf to 0.89967, saving model to model2nd_stacking.h5
roc_auc: 0.25811 - roc_auc_val: 0.28207 - norm_gini: -0.48379 - norm_gini_val: -0.43586          
Epoch 2/1000
 - 1s - loss: 0.5050 - acc: 0.8492 - val_loss: 0.4438 - val_acc: 0.8998

Epoch 00002: val_acc improved from 0.89967 to 0.89980, saving model to model2nd_stacking.h5
roc_auc: 0.82232 - roc_auc_val: 0.82656 - norm_gini: 0.64464 - norm_gini_val: 0.65312          
Epoch 3/1000
 - 1s - loss: 0.4060 - acc: 0.8812 - val_loss: 0.3834 - val_acc: 0.8995

Epoch 00003: val_acc did not improve from 0.89980
roc_auc: 0.83939 - roc_auc_val: 0.84268 - norm_gini: 0.67879 - norm_gini_val: 0.68537          
Epoch 4/1000
 - 1s - loss: 0.3521 - acc: 0.8977 - val_loss: 0.3409 - val_acc: 0.8995

Epoch 00004: val_acc did not improve from 0.89980
roc_auc: 0.84686 - roc_auc_val: 0.8


Epoch 00034: val_acc did not improve from 0.91336
roc_auc: 0.86577 - roc_auc_val: 0.86654 - norm_gini: 0.73154 - norm_gini_val: 0.73307          
Epoch 35/1000
 - 1s - loss: 0.2379 - acc: 0.9123 - val_loss: 0.2354 - val_acc: 0.9128

Epoch 00035: val_acc did not improve from 0.91336
roc_auc: 0.86584 - roc_auc_val: 0.86667 - norm_gini: 0.73168 - norm_gini_val: 0.73334          
Epoch 36/1000
 - 1s - loss: 0.2382 - acc: 0.9122 - val_loss: 0.2354 - val_acc: 0.9122

Epoch 00036: val_acc did not improve from 0.91336
roc_auc: 0.86574 - roc_auc_val: 0.86649 - norm_gini: 0.73147 - norm_gini_val: 0.73298          
Epoch 37/1000
 - 1s - loss: 0.2382 - acc: 0.9122 - val_loss: 0.2353 - val_acc: 0.9131

Epoch 00037: val_acc did not improve from 0.91336
roc_auc: 0.86594 - roc_auc_val: 0.86677 - norm_gini: 0.73187 - norm_gini_val: 0.73355          
Epoch 38/1000
 - 1s - loss: 0.2380 - acc: 0.9123 - val_loss: 0.2353 - val_acc: 0.9126

Epoch 00038: val_acc did not improve from 0.91336
roc_auc: 0.86584 

roc_auc: 0.86622 - roc_auc_val: 0.86705 - norm_gini: 0.73245 - norm_gini_val: 0.7341          
Epoch 70/1000
 - 1s - loss: 0.2378 - acc: 0.9118 - val_loss: 0.2347 - val_acc: 0.9120

Epoch 00070: val_acc did not improve from 0.91336
roc_auc: 0.86628 - roc_auc_val: 0.86716 - norm_gini: 0.73256 - norm_gini_val: 0.73432          
Epoch 71/1000
 - 1s - loss: 0.2381 - acc: 0.9111 - val_loss: 0.2346 - val_acc: 0.9120

Epoch 00071: val_acc did not improve from 0.91336
roc_auc: 0.86629 - roc_auc_val: 0.86712 - norm_gini: 0.73258 - norm_gini_val: 0.73425          
Epoch 72/1000
 - 1s - loss: 0.2374 - acc: 0.9118 - val_loss: 0.2346 - val_acc: 0.9123

Epoch 00072: val_acc did not improve from 0.91336
roc_auc: 0.8663 - roc_auc_val: 0.8672 - norm_gini: 0.7326 - norm_gini_val: 0.73439          
Epoch 73/1000
 - 1s - loss: 0.2374 - acc: 0.9126 - val_loss: 0.2346 - val_acc: 0.9128

Epoch 00073: val_acc did not improve from 0.91336
roc_auc: 0.86647 - roc_auc_val: 0.86735 - norm_gini: 0.73293 - norm_gini

#### 2ndModel XGB

In [198]:
XGB_2nd_B = xgb.DMatrix(part_B_X, part_B_target)
XGB_2nd_C = xgb.DMatrix(part_C_X, part_C_target)


In [217]:
params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": 0.01,
        "tree_method": 'exact',
        "max_depth": 22,
        "subsample": 1,
        "colsample_bytree": 1,
        "silent": 1,
        "min_chil_weight":1,
        "seed": 42,
        #"num_class" : 22,
    }
num_boost_round = 10000
early_stopping_rounds = 100
watchlist = [(XGB_2nd_B, 'train'), (XGB_2nd_C, 'eval')]

In [218]:
gbm = xgb.train(params, XGB_2nd_B, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)


[0]	train-auc:0.89133	eval-auc:0.876571
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 100 rounds.
[1]	train-auc:0.891706	eval-auc:0.876687
[2]	train-auc:0.892448	eval-auc:0.877205
[3]	train-auc:0.89376	eval-auc:0.877809
[4]	train-auc:0.89388	eval-auc:0.877709
[5]	train-auc:0.894445	eval-auc:0.877636
[6]	train-auc:0.894673	eval-auc:0.877528
[7]	train-auc:0.894977	eval-auc:0.877391
[8]	train-auc:0.895431	eval-auc:0.877303
[9]	train-auc:0.895657	eval-auc:0.877296
[10]	train-auc:0.895866	eval-auc:0.877312
[11]	train-auc:0.896184	eval-auc:0.877272
[12]	train-auc:0.896281	eval-auc:0.877239
[13]	train-auc:0.896683	eval-auc:0.877191
[14]	train-auc:0.897163	eval-auc:0.877146
[15]	train-auc:0.897285	eval-auc:0.877234
[16]	train-auc:0.89779	eval-auc:0.87725
[17]	train-auc:0.897995	eval-auc:0.877244
[18]	train-auc:0.898169	eval-auc:0.877197
[19]	train-auc:0.898552	eval-auc:0.877174
[20]	train-auc:0.898877	eval-auc:

In [190]:
y_pred_Xtest_XGB.columns=['NN_predict', 'XGB_predict']

In [206]:
y_pred = gbm.predict(xgb.DMatrix(y_pred_Xtest_XGB))

In [207]:
test['target'] = y_pred

In [208]:
test.loc[:,['ID_code','target']].to_csv('submission_stacking_with_xgb.csv', index=False)